# Ejercicio Joins  Filter
Obtener información de películas del 2000 en adelante ordenado de manera descendente por la fecha de lanzamiento, se debe considerar su respectivo "género" e "idioma" que se encuentra dicha película.

- De la película se requiere motrar el "título", "tiempo de duración", "fecha de lanzamiento", "voto promedio".
- De idioma se requiere mostrar el "nombre del idioma".
- Del género mostrar el "nombre del género".

In [0]:
%run "../includes/configuration"

In [0]:
movie_df = spark.read.parquet(f"{silver_folder_path}/movies").filter("year_release_date > 2000") \
    .select("movie_id","title", "duration_time", "release_date", "vote_average")

In [0]:
movie_languages = spark.read.parquet(f"{silver_folder_path}/movies_languages")

In [0]:
languages_df = spark.read.parquet(f"{silver_folder_path}/languages")

In [0]:
genre_df = spark.read.parquet(f"{silver_folder_path}/genres")

In [0]:
movie_genre_df = spark.read.parquet(f"{silver_folder_path}/movies_genres")

### Join language y movie_languages

In [0]:
language_mov_lan = languages_df.join(movie_languages, movie_languages.language_id == languages_df.language_id, "inner") \
    .select(languages_df.language_name, movie_languages.movie_id)

In [0]:
display(language_mov_lan)

### Join genre y movie_genres

In [0]:
genre_mov_gen = genre_df.join(movie_genre_df, movie_genre_df.genre_id == genre_df.genre_id, "inner") \
    .select(genre_df.genre_name, movie_genre_df.movie_id)

## Join movie_df, language_mov_lan, genre_mov_gen

In [0]:
movie_filtered_df = movie_df.filter("year_release_date > 2000")

In [0]:
results_mov_gen_lan_df = movie_filtered_df.join(language_mov_lan, language_mov_lan.movie_id == movie_filtered_df.movie_id, "inner").join(genre_mov_gen, genre_mov_gen.movie_id == movie_filtered_df.movie_id, "inner").orderBy("release_date", ascending=False) #Ordenamiento

- Agregar columna "created_date"

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
results_df = results_mov_gen_lan_df.select("title","duration_time" ,"release_date","vote_average", "language_name", "genre_name").withColumn("created_date", current_timestamp())

In [0]:
display(results_df)

### Escribir datos en formato parquet en el datalake

In [0]:
# results_df.write.mode("overwrite").parquet(f"{gold_folder_path}/results_movie_genre_language")
results_df.write.mode("overwrite").format('parquet').saveAsTable("movie_gold.results_movie_genre_language")

In [0]:
display(spark.read.parquet(f"{gold_folder_path}/results_movie_genre_language"))